In [149]:
# This reloads the extension if already loaded - Everytime you update the .py files, re-run this cell!
%reload_ext autoreload
# Automatically reloads modules before executing code OR makes Jupyter reload your .py files whenever you run a cell.
%autoreload 2

In [151]:
# Importing all of the required python packages
import sys
import os

In [153]:
# Setting up the folder paths.
sys.path.append('./lib')

# Import resuable code (scripts) within .py files.
from data_loader import load_csv
from data_wrangle import remove_cols

In [155]:
df = load_csv('./data/raw/U.S._Chronic_Disease_Indicators.csv')
print(df.head())

   YearStart  YearEnd LocationAbbr   LocationDesc     DataSource  \
0       2020     2020           US  United States          BRFSS   
1       2015     2019           AR       Arkansas  US Cancer DVT   
2       2015     2019           CA     California  US Cancer DVT   
3       2015     2019           CO       Colorado  US Cancer DVT   
4       2015     2019           GA        Georgia  US Cancer DVT   

           Topic                                           Question  Response  \
0  Health Status            Recent activity limitation among adults       NaN   
1         Cancer    Invasive cancer (all sites combined), incidence       NaN   
2         Cancer  Cervical cancer mortality among all females, u...       NaN   
3         Cancer    Invasive cancer (all sites combined), incidence       NaN   
4         Cancer  Prostate cancer mortality among all males, und...       NaN   

  DataValueUnit      DataValueType  ...  TopicID  QuestionID ResponseID  \
0        Number  Age-adjusted

In [ ]:
df2 = load_csv('./data/raw/US_Census_Data_2022_v01.csv')
print(df2.head())

In [ ]:
clean_df = remove_cols(df)
print(clean_df.head())